In [1]:
from datasets import load_dataset
 
# Convert dataset to OAI messages
system_message = """You are an text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.
SCHEMA:
{schema}"""
 
def create_conversation(sample):
  return {
    "messages": [
      {"role": "system", "content": system_message.format(schema=sample["context"])},
      {"role": "user", "content": sample["question"]},
      {"role": "assistant", "content": sample["answer"]}
    ]
  }
 
# Load dataset from the hub
dataset = load_dataset("b-mc2/sql-create-context", split="train")
dataset = dataset.shuffle().select(range(12500))
 
# Convert dataset to OAI messages
dataset = dataset.map(create_conversation, remove_columns=dataset.features,batched=False)
# split dataset into 10,000 training samples and 2,500 test samples
dataset = dataset.train_test_split(test_size=2500/12500)
 
print(dataset["train"][345]["messages"])
 
# save datasets to disk
dataset["train"].to_json("train_dataset.json", orient="records")
dataset["test"].to_json("test_dataset.json", orient="records")

README.md:   0%|          | 0.00/4.43k [00:00<?, ?B/s]

Map:   0%|          | 0/12500 [00:00<?, ? examples/s]

[{'content': 'You are an text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.\nSCHEMA:\nCREATE TABLE table_22347090_6 (directed_by VARCHAR, us_viewers__million_ VARCHAR)', 'role': 'system'}, {'content': 'How many directors are there for the episode that had 1.59 million U.S. viewers?', 'role': 'user'}, {'content': 'SELECT COUNT(directed_by) FROM table_22347090_6 WHERE us_viewers__million_ = "1.59"', 'role': 'assistant'}]


Creating json from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

1189717

In [2]:
# Load jsonl data from disk
dataset = load_dataset("json", data_files="train_dataset.json", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
dataset

Dataset({
    features: ['messages'],
    num_rows: 10000
})

In [4]:
dataset['messages']

[[{'content': 'You are an text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.\nSCHEMA:\nCREATE TABLE table_name_72 (pick INTEGER, player_name VARCHAR, year VARCHAR)',
   'role': 'system'},
  {'content': 'What pick was mario williams before 2006?', 'role': 'user'},
  {'content': 'SELECT SUM(pick) FROM table_name_72 WHERE player_name = "mario williams" AND year < 2006',
   'role': 'assistant'}],
 [{'content': 'You are an text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.\nSCHEMA:\nCREATE TABLE table_name_76 (date VARCHAR, runner_s__up VARCHAR)',
   'role': 'system'},
  {'content': 'What date did Ian poulter become runner up?', 'role': 'user'},
  {'content': 'SELECT date FROM table_name_76 WHERE runner_s__up = "ian poulter"',
   'role': 'assistant'}],
 [{'content': 'You are an text to SQL query translator. Users will ask you q